In [21]:
import pandas as pd 

In [22]:
df=pd.read_csv('WELFake_Dataset.csv')

In [23]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [24]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [25]:
#all null values are removed
df=df.dropna()

In [26]:
#Independent features
X=df.drop('label',axis=1)

In [27]:
y=df['label']

In [28]:
import tensorflow as tf

In [29]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [30]:
voc_size=5000

In [31]:
messages=X.copy()

In [39]:
messages.reset_index(inplace=True)

In [40]:
messages

,index,Unnamed: 0,title,text
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...,...
71532,72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
71533,72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
71534,72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
71535,72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...


In [41]:
messages['title'].iloc[0] 

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [42]:
import nltk
import re
from nltk.corpus import stopwords

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\TUSHAR
[nltk_data]     GUPTA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4761, 1894, 3491, 3205, 1154, 574, 2384, 1556, 2399, 3840, 4911, 1121],
 [4957,
  3904,
  3796,
  3122,
  3995,
  1118,
  2540,
  30,
  1655,
  4717,
  2179,
  568,
  1072,
  1121],
 [1126, 4905, 819, 4569, 4178, 424, 1899, 1680, 1903, 2117, 2158, 143],
 [1322, 1075, 2937, 4936, 1149, 405, 4821, 1935, 3664, 3460, 1213],
 [4541, 1899, 3033, 3875, 3116, 4803, 4623, 1617, 3033],
 [1024, 4799, 3434, 2454, 4832, 3535, 2781, 1929, 2628, 3399, 1979],
 [46, 2558, 3260, 1264, 1075, 1237, 424, 1867, 1498, 1121],
 [1777,
  35,
  3199,
  2701,
  1968,
  3038,
  881,
  2568,
  1182,
  1777,
  504,
  2855,
  2210,
  2235,
  1121],
 [3425, 2760, 2220, 3205, 1405, 1511, 1403, 2760],
 [3121, 2028, 3479, 271, 2862, 1458, 898, 1396],
 [2927, 4255, 3791, 1656, 2738, 4169, 2657, 4760, 2657, 3431],
 [2951, 2553, 1264, 3927, 4181, 505, 199, 1738, 2229],
 [183, 2250, 3424, 2553, 2117, 4909, 4407, 3824, 3059, 1558, 2050],
 [1986, 1251, 3718, 4628, 2442, 4462, 857, 633, 1841, 916],
 [106, 1461, 3012, 1052, 39

In [46]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4761 1894 3491 ...    0    0    0]
 [4957 3904 3796 ...    0    0    0]
 [1126 4905  819 ...    0    0    0]
 ...
 [ 958  592 4090 ...    0    0    0]
 [1264 1608 1896 ...    0    0    0]
 [3824 1660 3090 ...    0    0    0]]


In [47]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

C:\Users\TUSHAR GUPTA\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [48]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [50]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8023 - loss: 0.4038 - val_accuracy: 0.8926 - val_loss: 0.2562
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9070 - loss: 0.2246 - val_accuracy: 0.8939 - val_loss: 0.2570
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9210 - loss: 0.1989 - val_accuracy: 0.8938 - val_loss: 0.2551
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9288 - loss: 0.1777 - val_accuracy: 0.8937 - val_loss: 0.2613
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9412 - loss: 0.1552 - val_accuracy: 0.8894 - val_loss: 0.2874
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9491 - loss: 0.1334 - val_accuracy: 0.8944 - val_loss: 0.2898
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9581 - loss: 0.1051 - val_accuracy: 0.8931 - val_loss: 0.3102
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9664 - loss: 0.0881 - val_ac

In [58]:
y_pred=model.predict(X_test)

738/738 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [59]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
confusion_matrix(y_test,y_pred)

array([[10400,  1293],
       [ 1280, 10635]], dtype=int64)

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8910115215181295

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89     11693
           1       0.89      0.89      0.89     11915

    accuracy                           0.89     23608
   macro avg       0.89      0.89      0.89     23608
weighted avg       0.89      0.89      0.89     23608

